<a href="https://colab.research.google.com/github/keerthi5465/GENAI/blob/main/GenAI_CA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
import PyPDF2
from langchain_groq import ChatGroq


GROQ_API_KEY = ""
MODEL = "llama3-8b-8192"

llm = ChatGroq(api_key=GROQ_API_KEY, model=MODEL)

def extract_text_from_pdf(pdf_file):
    try:
        reader = PyPDF2.PdfReader(pdf_file)
        text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
        return text if text else "Could not extract text from PDF."
    except Exception as e:
        return f"Error extracting text: {str(e)}"

def generate_interview_questions(resume_text, domain):
    prompt = (
        f"Analyze the following resume:\n\n{resume_text}\n\n"
        f"Based on this resume, generate 5 relevant interview questions for a {domain} job role."
    )

    try:
        response = llm.invoke(prompt)
        return response.content if response else "Error generating questions."
    except Exception as e:
        return f"Error: {str(e)}"

def score_resume(resume_text, job_description):
    prompt = (
        f"Analyze the following resume:\n\n{resume_text}\n\n"
        f"Compare it with the job description:\n\n{job_description}\n\n"
        f"Provide a match percentage score (out of 100) along with key strengths and improvement areas."
    )

    try:
        response = llm.invoke(prompt)
        return response.content if response else "Error scoring resume."
    except Exception as e:
        return f"Error: {str(e)}"

with gr.Blocks() as app:
    gr.Markdown("# 📄 Resume Analyzer & Interview Question Generator")

    with gr.Row():
        resume_file = gr.File(label="Upload Resume (PDF)")
        domain = gr.Textbox(label="Enter Target Job Role (e.g., Data Scientist, Software Engineer)")

    with gr.Row():
        job_desc = gr.Textbox(label="Paste Job Description for Scoring (Optional)", lines=5)

    generate_btn = gr.Button("Analyze Resume")
    interview_output = gr.Textbox(label="Generated Interview Questions", interactive=False)
    score_output = gr.Textbox(label="Resume Score & Feedback", interactive=False)

    def analyze_resume(resume_file, domain, job_desc):
        resume_text = extract_text_from_pdf(resume_file.name)
        if "Error" in resume_text:
            return resume_text, resume_text

        questions = generate_interview_questions(resume_text, domain)
        score = score_resume(resume_text, job_desc) if job_desc else "No job description provided."
        return questions, score

    generate_btn.click(analyze_resume, inputs=[resume_file, domain, job_desc], outputs=[interview_output, score_output])

app.launch()


In [ ]:
import gradio as gr
from langchain_groq import ChatGroq


groq_api_key = ""
Model = "llama-3.1-8b-instant"
llm = ChatGroq(api_key=groq_api_key, model=Model)

def query_groq(prompt):
    """Calls the Groq API and returns a response."""
    try:
        response = llm.invoke(prompt)
        return response.content if response else "Error generating response."
    except Exception as e:
        return f"Error: {str(e)}"

def find_learning_resources(topic):
    """Finds relevant learning resources for a given topic."""
    prompt = (
        f"Suggest high-quality learning resources for {topic}, including:\n"
        "- YouTube videos (with links if possible)\n"
        "- Online courses (free & paid)\n"
        "- Articles & blogs\n"
        "- Free PDFs or books\n"
        "- Official documentation\n"
        "Ensure the recommendations are beginner-friendly if needed."
    )
    return query_groq(prompt)


with gr.Blocks() as app:
    gr.Markdown("# 📚 AI Learning Resource Finder")

    topic_input = gr.Textbox(label="Enter a Topic (e.g., Data Structures, AI, Cybersecurity)")
    resources_output = gr.Textbox(label="Learning Resources", interactive=False)
    search_resources_btn = gr.Button("Find Resources")

    search_resources_btn.click(find_learning_resources, inputs=[topic_input], outputs=resources_output)

app.launch()
